In [24]:
import numpy as np

In [5]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [5]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))

In [ ]:
def cross_entropy_error_batch(y, t):
    if y.ndim == 1:
        t = np.reshape(1,t.size)
        y = np.reshape(1,y.size)
        
    batch_size = y.shape[0]
    delta = 1e-7
    return -np.sum(t*np.log(y+delta)) / batch_size

In [6]:
# 편미분과 기울기

def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = fxh1 - fxh2 / (2*h)
        x[idx] = tmp_val
        
        it.iternext()
        
    return grad
    

In [10]:
# 경사하강법

def gradient_descent(f, init_x, lr=0.01, step_num=100):
    
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
        
    return x


In [32]:
class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.rand(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.rand(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def sigmoid(self, x):
        return 1 / (1+np.exp(-x))
        
    def softmax(self, x):
        c = np.max(x)
        exp_x = np.exp(x)
        exp_all = np.sum(exp_x)
        return exp_x / exp_all
    
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = self.softmax(a1)
        a2 = np.dot(z1, W2) + b2
        y = self.softmax(a2)
        
        return y
    
    def cross_entropy_error(self, y, t):
        
        if y.ndim == 1:
            t = t.reshape(1, t.size)
            y = y.reshape(1, y.size)
            
        if y.ndim == t.ndim:
            t = t.argmax(axis=1)
            
        batch_size = y.shape[0]
        
        return -np.sum(np.log(y[np.arange(batch_size),t])) / batch_size
       
        
    def loss(self, x, t):
        
        y = self.predict(x)
        
        return self.cross_entropy_error(y,t)
        
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
       
        
    def numerical_gradient(self, f, x):
        h = 1e-4
        grad = np.zeros_like(x)
        
        it = np.nditer(x, flags =['multi_index'], op_flags=['readwrite'])
        
        while not it.finished:
            idx = it.multi_index
            tmp_val = x[idx]
            
            x[idx] = tmp_val + h
            fxh1 = f(x)
            
            x[idx]  = tmp_val - h
            fxh2 = f(x)
            
            grad[idx] = (fxh1 - fxh2) / (2*h)
            x[idx] = tmp_val
            
            it.iternext()
            
        return grad
        
        
    def numerical_gradient_all(self, x, t):
        
        loss_W = lambda W: self.loss(x,t)
        grads = {}
        
        grads['W1'] = self.numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = self.numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = self.numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = self.numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
        


In [33]:
from dataset03.dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

In [34]:
iters_num = 1000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size = 784, hidden_size=50, output_size = 10)

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size/batch_size, 1)

In [ ]:
from tqdm.auto import tqdm # progress bar

for i in tqdm(range(iters_num)):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient_all(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ['W1','b1','W2', 'b2']:
        grad[key] -= learning_rate * grad[key]
    
     # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
     # 1에폭 당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | "+ str(train_acc) + ", " + str(test_acc))

  0%|          | 0/1000 [00:00<?, ?it/s]